In [ ]:
import datetime as dt

from seabeepy.config import SETTINGS
from pathlib import Path

import seabeepy as sb
import requests
import json

In [ ]:
def get_datasets(url) -> list[dict]:
    resp = requests.get(url).json()
    datasets = resp["datasets"]
    if resp["links"]["next"] is not None:
        datasets.extend(get_datasets(resp["links"]["next"]))

    return datasets

In [ ]:
geonode_url = "https://geonode.seabee.sigma2.no"
datasets = get_datasets(f"{geonode_url}/api/v2/datasets")

In [ ]:
datasets = [ds for ds in datasets if ds["subtype"] == "raster" and "niva" in ds["abstract"].lower()]
len(datasets)

In [ ]:
dataset_no_year = [ds for ds in datasets if ds["subtype"] == "raster" and "niva-tidy/niva" in ds["abstract"].lower()]

In [ ]:
len(dataset_no_year)

In [ ]:
minio_path_lookup = {}
for ds in dataset_no_year:
    for s in ds["raw_abstract"].split(" "):
        if s.startswith("niva-tidy"):
            minio_path_lookup[ds['name']] = s.replace("niva-tidy", "niva-tidy/2023")

In [ ]:
for layer_name, minio_path in minio_path_lookup.items():
    path = Path(f"/home/notebook/shared-seabee-ns9879k/{minio_path}")
    if path.exists() and (path / "orthophoto"/ f"{layer_name}.tif").exists():
        print(f"Updating {layer_name}")
        abstract = sb.geo.get_html_abstract(str(path))
        date = sb.ortho.parse_mission_data(path, parse_date=True)[2]
        metadata = {
            "abstract": abstract,
            "date": date.isoformat(),
            "date_type": "creation",
            "attribution": "SeaBee",
        }
        sb.geo.update_geonode_metadata(
            layer_name,
            SETTINGS.GEONODE_USER,
            SETTINGS.GEONODE_PASSWORD,
            metadata,
        )

In [ ]:
datasets_seabee_data = [ds for ds in datasets if ds["subtype"] == "raster" and "SEABEE_DATA" in ds["abstract"]]

In [ ]:
len(datasets_seabee_data)

In [ ]:
for ds in datasets_seabee_data:
    print(ds["title"])

In [ ]:
minio_path_lookup_seabee = {}
for ds in datasets_seabee_data:
    for s in ds["raw_abstract"].split(" "):
        if s.startswith("niva-tidy"):
            minio_path_lookup_seabee[ds['name']] = s.replace("SEABEE_DATA/", "")

In [ ]:
for layer_name, minio_path in minio_path_lookup_seabee.items():
    path = Path(f"/home/notebook/shared-seabee-ns9879k/{minio_path}")
    if path.exists() and (path / "orthophoto"/ f"{layer_name}.tif").exists():
        print(f"Updating {layer_name}")
        abstract = sb.geo.get_html_abstract(str(path))
        date = sb.ortho.parse_mission_data(path, parse_date=True)[2]
        metadata = {
            "abstract": abstract,
            "date": date.isoformat(),
            "date_type": "creation",
            "attribution": "SeaBee",
        }
        sb.geo.update_geonode_metadata(
            layer_name,
            SETTINGS.GEONODE_USER,
            SETTINGS.GEONODE_PASSWORD,
            metadata,
        )